In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import data_loader
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from statsmodels.api import OLS
from xgboost import XGBRegressor

In [25]:
df = data_loader.load_data()
df

avg_income  avg_prop_tax  \
statefip Date                                      
Alabama  2000-01-31   41917.384970    271.925118   
         2000-02-29   41917.384970    271.925118   
         2000-03-31   41917.384970    271.925118   
         2000-04-30   41917.384970    271.925118   
         2000-05-31   41917.384970    271.925118   
...                            ...           ...   
Wyoming  2024-09-30  104455.891107      0.000000   
         2024-10-31  104455.891107      0.000000   
         2024-11-30  104455.891107      0.000000   
         2024-12-31  104455.891107      0.000000   
         2025-01-31  104455.891107      0.000000   

                     proportion_American Indian/Aleut/Eskimo  \
statefip Date                                                  
Alabama  2000-01-31                                 0.000000   
         2000-02-29                                 0.000000   
         2000-03-31                                 0.000000   
         2000-04-30                                 0.000000   
         2000-05-31                                 0.000000   
...                                                      ...   
Wyoming  2024-09-30                                 0.008195   
         2024-10-31                                 0.008195   
         2024-11-30                                 0.008195   
         2024-12-31                                 0.008195   
         2025-01-31                                 0.008195   

                     proportion_Asian only  proportion_Black  \
statefip Date                                                  
Alabama  2000-01-31               0.000000          0.262985   
         2000-02-29               0.000000          0.262985   
         2000-03-31               0.000000          0.262985   
         2000-04-30               0.000000          0.262985   
         2000-05-31               0.000000          0.262985   
...                                    ...               ...   
Wyoming  2024-09-30               0.007245          0.007625   
         2024-10-31               0.007245          0.007625   
         2024-11-30               0.007245          0.007625   
         2024-12-31               0.007245          0.007625   
         2025-01-31               0.007245          0.007625   

                     proportion_White  total_population  Median Home Price  
statefip Date                                                               
Alabama  2000-01-31          0.737015         706842.48       97080.984792  
         2000-02-29          0.737015         706842.48       97250.037728  
         2000-03-31          0.737015         706842.48       97363.306485  
         2000-04-30          0.737015         706842.48       97617.062414  
         2000-05-31          0.737015         706842.48       97900.085921  
...                               ...               ...                ...  
Wyoming  2024-09-30          0.961807          67792.43      341148.869753  
         2024-10-31          0.961807          67792.43      343849.922709  
         2024-11-30          0.961807          67792.43      346044.040121  
         2024-12-31          0.961807          67792.43      348280.758951  
         2025-01-31          0.961807          67792.43      350224.417540  

[15351 rows x 8 columns]

In [22]:
X_cols = df.columns[1:-1]
y_col = df.columns[-1]

In [51]:
for group, data in df.groupby('statefip'):
    print(f"State: {group}")
    X = data[X_cols]
    X = sm.add_constant(X)
    print(data[y_col].name)
    y = np.log(data[y_col]).rename(f"log({data[y_col].name})")
    
    model = OLS(y, X).fit()
    print(model.summary())

    # Lag of 1 year back in time
    X['lag_1']= 0
    X['lag_1'][12:] = y.iloc[:-12].values
    model = OLS(y, X).fit()
    print(model.summary())

State: Alabama
Median Home Price
                              OLS Regression Results                              
Dep. Variable:     log(Median Home Price)   R-squared:                       0.568
Model:                                OLS   Adj. R-squared:                  0.560
Method:                     Least Squares   F-statistic:                     64.52
Date:                    Sat, 22 Mar 2025   Prob (F-statistic):           8.26e-51
Time:                            14:30:48   Log-Likelihood:                 145.05
No. Observations:                     301   AIC:                            -276.1
Df Residuals:                         294   BIC:                            -250.2
Df Model:                               6                                         
Covariance Type:                nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------

/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_48513/2499356049.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['lag_1'][12:] = y.iloc[:-12].values
/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_48513/2499356049.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['lag_1'][12:] = y.iloc[:-12].values
/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_48513/2499356049.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

                              OLS Regression Results                              
Dep. Variable:     log(Median Home Price)   R-squared:                       0.574
Model:                                OLS   Adj. R-squared:                  0.565
Method:                     Least Squares   F-statistic:                     65.96
Date:                    Sat, 22 Mar 2025   Prob (F-statistic):           1.33e-51
Time:                            14:30:48   Log-Likelihood:                 109.32
No. Observations:                     301   AIC:                            -204.6
Df Residuals:                         294   BIC:                            -178.7
Df Model:                               6                                         
Covariance Type:                nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------

/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_48513/2499356049.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['lag_1'][12:] = y.iloc[:-12].values
/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_48513/2499356049.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['lag_1'][12:] = y.iloc[:-12].values
/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_48513/2499356049.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

                              OLS Regression Results                              
Dep. Variable:     log(Median Home Price)   R-squared:                       0.840
Model:                                OLS   Adj. R-squared:                  0.836
Method:                     Least Squares   F-statistic:                     219.2
Date:                    Sat, 22 Mar 2025   Prob (F-statistic):          1.84e-112
Time:                            14:30:48   Log-Likelihood:                 162.25
No. Observations:                     301   AIC:                            -308.5
Df Residuals:                         293   BIC:                            -278.8
Df Model:                               7                                         
Covariance Type:                nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------

/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_48513/2499356049.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['lag_1'][12:] = y.iloc[:-12].values
/var/folders/10/d1ck4wk17fb0b8y0xq2l46w00000gn/T/ipykernel_48513/2499356049.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['lag_1'][12:] = y.iloc[:-12].values


In [61]:
params = []

for group, data in df.groupby('statefip'):
    # print(f"State: {group}")
    X = data[X_cols]
    X = sm.add_constant(X)
    # y = data[y_col]
    y = np.log(data[y_col]).rename(f"log({data[y_col].name})")
    
    model = OLS(y, X).fit()
    params.append(model.params)

print(pd.DataFrame(params).mean())
print(pd.DataFrame(params).std())

const                                      14.149201
avg_prop_tax                               -0.000198
proportion_American Indian/Aleut/Eskimo    -4.404952
proportion_Asian only                       8.175773
proportion_Black                           -0.835422
proportion_White                           -2.104818
total_population                            0.000001
dtype: float64
const                                      11.970856
avg_prop_tax                                0.000137
proportion_American Indian/Aleut/Eskimo    17.857954
proportion_Asian only                      18.119221
proportion_Black                           12.903663
proportion_White                           12.292423
total_population                            0.000003
dtype: float64


In [63]:
X = df[X_cols]
y = np.log(df[y_col])
X = sm.add_constant(X)

model = OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      Median Home Price   R-squared:                       0.210
Model:                            OLS   Adj. R-squared:                  0.210
Method:                 Least Squares   F-statistic:                     681.5
Date:                Sat, 22 Mar 2025   Prob (F-statistic):               0.00
Time:                        14:49:21   Log-Likelihood:                -7668.5
No. Observations:               15351   AIC:                         1.535e+04
Df Residuals:                   15344   BIC:                         1.540e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===========================================================================================================
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
const                                      11.5101      0.094    121.950      0.000      11.325      11.695
avg_prop_tax                            -3.606e-05   3.19e-06    -11.301      0.000   -4.23e-05   -2.98e-05
proportion_American Indian/Aleut/Eskimo     1.3293      0.210      6.320      0.000       0.917       1.742
proportion_Asian only                       4.3894      0.150     29.172      0.000       4.094       4.684
proportion_Black                            0.6761      0.101      6.686      0.000       0.478       0.874
proportion_White                            0.5957      0.096      6.226      0.000       0.408       0.783
total_population                         2.835e-08   4.57e-09      6.207      0.000    1.94e-08    3.73e-08
==============================================================================
Omnibus:                      213.056   Durbin-Watson:                   0.023
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              202.700
Skew:                           0.248   Prob(JB):                     9.64e-45
Kurtosis:                       2.735   Cond. No.                     8.42e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.42e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""